In [2]:
from website.pipeline import training
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [2]:
raw = pd.read_json('data/data.json')
X_train, X_test, y_train, y_test = training(raw)

In [8]:
%run src/pipeline
# Read data from file:
D = FraudData('data/data.json')
df = D.get_enriched_df()
X_train, X_test, y_train, y_test = D.get_train_test_data()

X_train.drop(columns = 'label', inplace = True)
X_test.drop(columns = 'label', inplace = True)

/Users/tylerpreston/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,AUD,CAD,EUR,GBP,MXN,NZD,USD,body_length_0,body_length_100,body_length_200,...,channel_other,user_type_1,user_type_2,user_type_3,user_type_4,cost min,cost max,quantity_total sum,ticket_tiers_num,label
11821,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,65.0,65.0,200.0,1,0


In [9]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(X_train, y_train)

%time
y_pred = nb.predict(X_test)
print('accuracy %s' % accuracy_score(y_test,y_pred))
print('precision %s'% precision_score(y_test,y_pred))
print('recall %s'% recall_score(y_test,y_pred))
print(confusion_matrix(y_test, y_pred))

ValueError: Input X must be non-negative

In [10]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)

%time

y_pred = logreg.predict(X_test)
print('accuracy %s' % accuracy_score(y_test,y_pred))
print('precision %s'% precision_score(y_test,y_pred))
print('recall %s'% recall_score(y_test,y_pred))
print(confusion_matrix(y_test, y_pred))

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs
accuracy 0.9191073919107392
precision 0.6460176991150443
recall 0.2260061919504644
[[3222   40]
 [ 250   73]]


/Users/tylerpreston/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [11]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=.01,
                                 max_depth=3, random_state=29)
gbc.fit(X_train,y_train)
%time

y_pred = gbc.predict(X_test)
print('accuracy %s' % accuracy_score(y_test,y_pred))
print('precision %s'% precision_score(y_test,y_pred))
print('recall %s'% recall_score(y_test,y_pred))
print(confusion_matrix(y_test, y_pred))

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10 µs
accuracy 0.9207810320781032
precision 0.9333333333333333
recall 0.13003095975232198
[[3259    3]
 [ 281   42]]


In [12]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=1000,max_depth=3,random_state=29,
                            oob_score=True,n_jobs=-1)
rf.fit(X_train,y_train)
%time

y_pred = rf.predict(X_test)
print('accuracy %s' % accuracy_score(y_test,y_pred))
print('precision %s'% precision_score(y_test,y_pred))
print('recall %s'% recall_score(y_test,y_pred))
print(confusion_matrix(y_test, y_pred))

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs
accuracy 0.9182705718270572
precision 0.96875
recall 0.09597523219814241
[[3261    1]
 [ 292   31]]


In [13]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, 
                    random_state=42, max_iter=5, tol=None)

sgd.fit(X_train,y_train)
%time

y_pred = sgd.predict(X_test)
print('accuracy %s' % accuracy_score(y_test,y_pred))
print('precision %s'% precision_score(y_test,y_pred))
print('recall %s'% recall_score(y_test,y_pred))
print(confusion_matrix(y_test, y_pred))

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs
accuracy 0.24463040446304044
precision 0.0627062706270627
recall 0.5294117647058824
[[ 706 2556]
 [ 152  171]]


/Users/tylerpreston/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


In [14]:
import pickle
pickle.dump(logreg,open('website/logreg.pkl','wb'))